### Use rio-viz in Jupyter Notebook

In [1]:
import time
import httpx

from ipyleaflet import Map, ScaleControl, FullScreenControl, SplitMapControl, TileLayer

from rio_tiler.io import STACReader
from rio_viz.app import Client

In [2]:
# Create rio-viz Client (using server-thread to launch backgroud task)
client = Client(
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_34SGA_20200318_0_L2A",
    reader=STACReader,
    # By default STACReader min/max zoom is 0->24
    # Knowledge of the Sentinel-2 data tell us it's more 8->14
    minzoom=8,
    maxzoom=14,
)

# Gives some time for the server to setup
time.sleep(1)

# Check that client is running
print("Client is alive: ", client.server.is_alive())

Client is alive:  True


/Users/vincentsarago/Dev/vincentsarago/rasterio/rasterio/io.py:140: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  return writer(mempath, 'w+', driver=driver, width=width,
ERROR 4: `/vsimem/c74073e6-8449-4f78-9e5e-389b559826fb/c74073e6-8449-4f78-9e5e-389b559826fb.tif' not recognized as a supported file format.
ERROR 4: `/vsimem/58df3647-f31c-496f-94f7-e56fde7c62f4/58df3647-f31c-496f-94f7-e56fde7c62f4.tif' not recognized as a supported file format.


In [4]:
tilejson = httpx.get(
    f"{client.endpoint}/tilejson.json",
    params = {
        "assets": ["B04", "B03", "B02"],
        "rescale": "0,10000"
    }
).json()

bounds = ((client.bounds[0], client.bounds[2]), (client.bounds[1], client.bounds[3]))
center = ((client.bounds[1] + client.bounds[3]) / 2, (client.bounds[0] + client.bounds[2]) / 2)

layer = TileLayer(
    url=tilejson["tiles"][0],
    min_zoom=tilejson["minzoom"],
    max_zoom=tilejson["maxzoom"],
    bounds=bounds,
)


# Make the ipyleaflet map
m = Map(center=center, zoom=client.minzoom)
m.add_layer(layer)
m

Map(center=[32.00833055925221, 23.794854319372455], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [18]:
left_tilejson = httpx.get(
    f"{client.endpoint}/tilejson.json",
    params = {
        "assets": ["B04", "B03", "B02"],
        "rescale": "0,10000"
    }
).json()

right_tilejson = httpx.get(
    f"{client.endpoint}/tilejson.json",
    params = {
        "assets": ["B05", "B04", "B03"],
        "rescale": "0,10000"
    }
).json()

bounds = ((client.bounds[0], client.bounds[2]), (client.bounds[1], client.bounds[3]))
center = ((client.bounds[1] + client.bounds[3]) / 2, (client.bounds[0] + client.bounds[2]) / 2)

left = TileLayer(
    url=left_tilejson["tiles"][0],
    min_zoom=left_tilejson["minzoom"],
    max_zoom=left_tilejson["maxzoom"],
    bounds=bounds,
)

right = TileLayer(
    url=right_tilejson["tiles"][0],
    min_zoom=right_tilejson["minzoom"],
    max_zoom=right_tilejson["maxzoom"],
    bounds=bounds,

)

# Make the ipyleaflet map
m = Map(center=center, zoom=client.minzoom)
control = SplitMapControl(left_layer=left, right_layer=right)

m.add_control(control)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(FullScreenControl())
m

Map(center=[32.00833055925221, 23.794854319372455], controls=(ZoomControl(options=['position', 'zoom_in_text',…

ERROR 4: `/vsimem/935dc790-e673-44bf-8e4e-ca3331fda444/935dc790-e673-44bf-8e4e-ca3331fda444.tif' not recognized as a supported file format.
ERROR 4: `/vsimem/34dd160d-3737-4ed0-88da-b31abc1a2e62/34dd160d-3737-4ed0-88da-b31abc1a2e62.tif' not recognized as a supported file format.
ERROR 4: `/vsimem/53daa08b-a033-4b8d-9323-75f7db0ed228/53daa08b-a033-4b8d-9323-75f7db0ed228.tif' not recognized as a supported file format.
